# Ejercicio: Fragmentador de capítulos en un documento

Define extensiones personalizadas para los objetos `Span` y `Doc`:

- Cada `Span` (capítulo) debe tener:
  - Su número (`numero`)
  - Su título (`titulo`)
  - Su longitud en tokens (`longitud`)

- El objeto `Doc` debe tener:
  - Una lista de todos los capítulos (`capitulos`)

Crea un componente personalizado para añadir luego al pipeline, llamado `fragmentador_capitulos` que:

- Reciba un `Doc`
- Busque los capítulos utilizando una expresión regular, donde cada capítulo esté definido por un número seguido de su título.
- Cree un `Span` para cada capítulo con los atributos personalizados definidos.
- Guarde la lista de capítulos en el atributo `doc._.capitulos`.

In [10]:
import spacy
from spacy.tokens import Span, Doc
from spacy.language import Language
import re

# ------------------------------
# Extensiones personalizadas
# ------------------------------
Span.set_extension("numero", default=None, force=True)
Span.set_extension("titulo", default=None, force=True)
Span.set_extension("longitud", getter=lambda span: len(span), force=True)
Doc.set_extension("capitulos", default=[], force=True)

# ------------------------------
# Componente personalizado
# ------------------------------
@Language.component("fragmentador_capitulos")

def fragmentador_capitulos(doc):
    patron = re.compile(r"\n+(\d+)\s*\n+([^\n]+)")
    coincidencias = list(patron.finditer(doc.text))
    capitulos = []

    for i, match in enumerate(coincidencias):
        numero = int(match.group(1))
        titulo = match.group(2).strip()

        start_char = match.end()
        end_char = coincidencias[i + 1].start() if i + 1 < len(coincidencias) else len(doc.text)

        # Crear spans seguros, con alignment_mode
        span = doc.char_span(start_char, end_char, alignment_mode="contract")

        if span is None:
            continue  # O puedes hacer logging para investigar por qué falló

        span._.numero = numero
        span._.titulo = titulo
        capitulos.append(span)

    doc._.capitulos = capitulos
    return doc


# Ejercicio: Segmentación en capítulos del tercer libro de Harry Potter

- Define la variable `nlp` con el modelo `es_core_news_md`.
- Añade el componente que has definirdo al pipeline de spaCy. Respecto a este pipeline, en el resto de esta notebook no será necesario utilizar el NER predefinido, ya que nuestro interés será detectar otro tipo de entidades que nosotros mismos vamos a definir. 
- Carga el tercer libro de Harry Potter a Partir de la línea 37. 
- Procesa el texto del tercer libro de Harry Potter con `nlp`, y comprueba que el `Doc` resultante contenga todos los capítulos segmentados y anotados correctamente.

In [11]:
# Ruta al archivo (ajusta si está en otra carpeta)
ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt"

# Leer el archivo desde la línea 38 en adelante
with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto = "".join(lineas[37:])  # índice 37 = línea 38 (empieza desde 0)



<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\uo294249\AppData\Local\Temp\ipykernel_13268\4147096685.py:2: SyntaxWarning: invalid escape sequence '\l'
  ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt"


In [12]:
nlp = spacy.load("es_core_news_md")

nlp.add_pipe("fragmentador_capitulos", last=True)
doc = nlp(texto)

In [13]:
for cap in doc._.capitulos:
    print(f"Capítulo {cap._.numero}: {cap._.titulo} ({cap._.longitud} tokens)")

Capítulo 1: Lechuzas mensajeras (4417 tokens)
Capítulo 2: El error de tía Marge (4650 tokens)
Capítulo 3: El autobús noctámbulo (5432 tokens)
Capítulo 4: El Caldero Chorreante (6464 tokens)
Capítulo 5: El dementor (8177 tokens)
Capítulo 6: Posos de té y garras de hipogrifo (8243 tokens)
Capítulo 7: El boggart del armario ropero (5463 tokens)
Capítulo 8: La huida de la señora gorda (6201 tokens)
Capítulo 9: La derrota (6387 tokens)
Capítulo 10: El mapa del merodeador (8670 tokens)
Capítulo 11: La Saeta de Fuego (6761 tokens)
Capítulo 12: El patronus (5808 tokens)
Capítulo 13: Gryffindor contra Ravenclaw (5151 tokens)
Capítulo 14: El rencor de Snape (6650 tokens)
Capítulo 15: La final de quidditch (6963 tokens)
Capítulo 16: La predicción de la profesora Trelawney (5274 tokens)
Capítulo 17: El perro, el gato y la rata (5102 tokens)
Capítulo 18: Lunático, Colagusano, Canuto y Cornamenta (2621 tokens)
Capítulo 19: El vasallo de lord Voldemort (6136 tokens)
Capítulo 20: El Beso del dementor 

## Ejercicio: Detectar hechizos y añadirlos como entidades

Utilizando el objeto `PhraseMatcher` de spaCy, crea un sistema que detecte menciones de hechizos en un texto y los añada como nuevas entidades. Utiliza la siguiente lista de hechizos:

`hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]`

Realiza este ejercicio de dos formas diferentes y razona qué está ocurriendo. 

1. Con el `texto` del tercer libro cargado tal cual
2. Haciendo la modificación `texto = texto.replace("", " ").replace("\x97", " ")` después de cargar el tercer libro.


In [14]:


from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]
id=0
for hechi in hechizos:
    id+=1
    pattern=nlp(hechi)
    matcher.add(f"patron_{id}", [pattern])
    
    
num=0
# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Obtén el span resultante
    num+=1
    span = doc[start:end]
    print(f"COINCIDENCIA {num}:", span.sent)
    print("--------------------------------")

COINCIDENCIA 1: Levantó la varita, murmuró ¡Lumos!, y vio que se encontraba en un pasadizo muy estrecho, bajo y cubierto de barro.
--------------------------------
COINCIDENCIA 2: El encantamiento es así Lupin se aclaró la garganta: ¡Expecto patronum!
¡Expecto patronum! repitió Harry entre dientes. ¡Expecto patronum!
¿Te estás concentrando con fuerza en el recuerdo feliz?
Sí... contestó Harry, obligando a su mente a que retrocediese hasta aquel primer viaje en escoba. Expecto patrono, no, patronum... perdón...
--------------------------------
COINCIDENCIA 3: El encantamiento es así Lupin se aclaró la garganta: ¡Expecto patronum!
¡Expecto patronum! repitió Harry entre dientes. ¡Expecto patronum!
¿Te estás concentrando con fuerza en el recuerdo feliz?
Sí... contestó Harry, obligando a su mente a que retrocediese hasta aquel primer viaje en escoba. Expecto patrono, no, patronum... perdón...
--------------------------------
COINCIDENCIA 4: ¡Expecto patronum! ¡Expecto pa

In [15]:
# Ruta al archivo (ajusta si está en otra carpeta)
ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt"

# Leer el archivo desde la línea 38 en adelante
with open(ruta_archivo, "r", encoding="utf-8") as f:
    lineas = f.readlines()
    texto = "".join(lineas[37:])  # índice 37 = línea 38 (empieza desde 0)

texto = texto.replace("", "-").replace("\x97", " ")



<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\uo294249\AppData\Local\Temp\ipykernel_13268\2333512826.py:2: SyntaxWarning: invalid escape sequence '\l'
  ruta_archivo = "datos2011\\trunk\libros\J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt"


In [16]:
nlp = spacy.load("es_core_news_md")

nlp.add_pipe("fragmentador_capitulos", last=True)
doc = nlp(texto)

In [17]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]
id=0
for hechi in hechizos:
    id+=1
    pattern=nlp(hechi)
    matcher.add(f"patron_{id}", [pattern])
    
    
num=0
# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Obtén el span resultante
    num+=1
    span = doc[start:end]
    print(f"COINCIDENCIA {num}:", span.sent)
    print("--------------------------------")

COINCIDENCIA 1: ¡Lumos!  susurró Harry.
--------------------------------
COINCIDENCIA 2: Levantó la varita, murmuró ¡Lumos!, y vio que se encontraba en un pasadizo muy estrecho, bajo y cubierto de barro.
--------------------------------
COINCIDENCIA 3: El encantamiento es así  Lupin se aclaró la garganta : ¡Expecto patronum!
 ¡Expecto patronum!  repitió Harry entre dientes .
--------------------------------
COINCIDENCIA 4: El encantamiento es así  Lupin se aclaró la garganta : ¡Expecto patronum!
 ¡Expecto patronum!  repitió Harry entre dientes .
--------------------------------
COINCIDENCIA 5: ¡Expecto patronum!
 ¿Te estás concentrando con fuerza en el recuerdo feliz?
 Sí...  contestó Harry, obligando a su mente a que retrocediese hasta aquel primer viaje en escoba .
--------------------------------
COINCIDENCIA 6: Expecto patrono, no, patronum... perdón... ¡Expecto patronum! ¡Expecto patronum!
De repente, como un chorro, surgió algo del extremo de su varita.
--------------------------

In [18]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]
id=0
for hechi in hechizos:
    id+=1
    pattern=nlp(hechi)
    matcher.add(f"patron_{id}", [pattern])
    
    
num=0
# Itera sobre los resultados
for match_id, start, end in matcher(doc):
    # Obtén el span resultante
    num+=1
    span = doc[start:end]
    print(f"COINCIDENCIA {num}:", span.sent)
    print("--------------------------------")

COINCIDENCIA 1: ¡Lumos!  susurró Harry.
--------------------------------
COINCIDENCIA 2: Levantó la varita, murmuró ¡Lumos!, y vio que se encontraba en un pasadizo muy estrecho, bajo y cubierto de barro.
--------------------------------
COINCIDENCIA 3: El encantamiento es así  Lupin se aclaró la garganta : ¡Expecto patronum!
 ¡Expecto patronum!  repitió Harry entre dientes .
--------------------------------
COINCIDENCIA 4: El encantamiento es así  Lupin se aclaró la garganta : ¡Expecto patronum!
 ¡Expecto patronum!  repitió Harry entre dientes .
--------------------------------
COINCIDENCIA 5: ¡Expecto patronum!
 ¿Te estás concentrando con fuerza en el recuerdo feliz?
 Sí...  contestó Harry, obligando a su mente a que retrocediese hasta aquel primer viaje en escoba .
--------------------------------
COINCIDENCIA 6: Expecto patrono, no, patronum... perdón... ¡Expecto patronum! ¡Expecto patronum!
De repente, como un chorro, surgió algo del extremo de su varita.
--------------------------

# CONJUNTO DE TRAIN Y TEST


> En el examen buscar entidades en un capitulo en contreto

In [ ]:
import spacy
from spacy.lang.es import Spanish

nlp=Spanish()

